# Fazendo uma nova DS
Pelo add data, consigo adicionar a competição rsna-intracranial-hemorrhage-detection ao notebook e acessar os dados sem precisar baixar!

## Imports e Instalações

In [1]:
!mkdir ~/.kaggle
!cp ../input/kagglejson/kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
import pandas as pd
import kaggle
import shutil

In [3]:
input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection"
train_images_dir = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/"
#!ls {input_path}

### Criando os diretorios

In [4]:
!mkdir small_rsna_ds
!mkdir small_rsna_ds/yes
!mkdir small_rsna_ds/no

## Selecionando os arquivos
*   https://www.kaggle.com/code/akensert/rsna-inceptionv3-keras-tf1-14-0?scriptVersionId=39570832 secção 4



### Definições

In [5]:
def read_trainset(filename=input_path+"/stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        56346, 56347, 56348, 56349,
        56350, 56351, 1171830, 1171831,
        1171832, 1171833, 1171834, 1171835,
        3705312, 3705313, 3705314, 3705315,
        3705316, 3705317, 3842478, 3842479,
        3842480, 3842481, 3842482, 3842483
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

df = read_trainset()

In [6]:
df

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000012eaf     0        0                0                0            0   
ID_000039fa0     0        0                0                0            0   
ID_00005679d     0        0                0                0            0   
ID_00008ce3c     0        0                0                0            0   
ID_0000950d7     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_ffff82e46     0        0                0                0            0   
ID_ffff922b9     1        0                0                1            0   
ID_ffffb670a     1        0                0                0            1   
ID_ffffcbff8     0        0                0                0            0   
ID_fffff9393     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_000012eaf        0  
ID_000039fa0        0  
ID_00005679d        0  
ID_00008ce3c        0  
ID_0000950d7        0  
...               ...  
ID_ffff82e46        0  
ID_ffff922b9        0  
ID_ffffb670a        0  
ID_ffffcbff8        0  
ID_fffff9393        0  

[752803 rows x 6 columns]

### Selecionando aleatoriamente

In [7]:
df_shuffled = df.sample(frac=1)
df_shuffled

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_50cbbf3af     0        0                0                0            0   
ID_33aa5866c     0        0                0                0            0   
ID_03094b4e2     0        0                0                0            0   
ID_14421962f     0        0                0                0            0   
ID_8a6fccf42     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_52ad191a2     0        0                0                0            0   
ID_2621d6acd     1        0                0                1            0   
ID_78d30f3f7     1        1                0                0            0   
ID_967f3a58e     0        0                0                0            0   
ID_696652087     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_50cbbf3af        0  
ID_33aa5866c        0  
ID_03094b4e2        0  
ID_14421962f        0  
ID_8a6fccf42        0  
...               ...  
ID_52ad191a2        0  
ID_2621d6acd        0  
ID_78d30f3f7        0  
ID_967f3a58e        0  
ID_696652087        0  

[752803 rows x 6 columns]

In [8]:
df_shorten = df_shuffled[:35000]
df_shorten

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_50cbbf3af     0        0                0                0            0   
ID_33aa5866c     0        0                0                0            0   
ID_03094b4e2     0        0                0                0            0   
ID_14421962f     0        0                0                0            0   
ID_8a6fccf42     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_9004866a0     0        0                0                0            0   
ID_60090d3be     0        0                0                0            0   
ID_8f1848824     1        0                0                0            1   
ID_59a66224c     0        0                0                0            0   
ID_ebeab4e80     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_50cbbf3af        0  
ID_33aa5866c        0  
ID_03094b4e2        0  
ID_14421962f        0  
ID_8a6fccf42        0  
...               ...  
ID_9004866a0        0  
ID_60090d3be        0  
ID_8f1848824        0  
ID_59a66224c        0  
ID_ebeab4e80        0  

[35000 rows x 6 columns]

## Criando o novo Dataset

In [9]:
n = 0
while n < 34999:
    id = df_shorten.index[n]
    hemorrhage = df.loc[[id]].iat[0,0]
    
    src = train_images_dir+id+'.dcm'
    
    if(hemorrhage == 1):
        dst = './small_rsna_ds/yes/'
    else:
        dst = './small_rsna_ds/no/'
    
    shutil.copy(src, dst)
    n = n+1

In [10]:
!zip -rm ds_35000.zip ./small_rsna_ds

  adding: small_rsna_ds/ (stored 0%)
  adding: small_rsna_ds/yes/ (stored 0%)
  adding: small_rsna_ds/yes/ID_4ff6f7bef.dcm (deflated 54%)
  adding: small_rsna_ds/yes/ID_6778be757.dcm (deflated 56%)
  adding: small_rsna_ds/yes/ID_53c31eabd.dcm (deflated 61%)
  adding: small_rsna_ds/yes/ID_67e686b91.dcm (deflated 58%)
  adding: small_rsna_ds/yes/ID_d36945bfd.dcm (deflated 62%)
  adding: small_rsna_ds/yes/ID_7612586e5.dcm (deflated 51%)
  adding: small_rsna_ds/yes/ID_5c3f5bf87.dcm (deflated 53%)
  adding: small_rsna_ds/yes/ID_013031e05.dcm (deflated 56%)
  adding: small_rsna_ds/yes/ID_115f878ce.dcm (deflated 49%)
  adding: small_rsna_ds/yes/ID_c5460c442.dcm (deflated 63%)
  adding: small_rsna_ds/yes/ID_e57d0e36b.dcm (deflated 60%)
  adding: small_rsna_ds/yes/ID_e70baf833.dcm (deflated 51%)
  adding: small_rsna_ds/yes/ID_24506c51d.dcm (deflated 58%)
  adding: small_rsna_ds/yes/ID_2b6d1f00d.dcm (deflated 54%)
  adding: small_rsna_ds/yes/ID_b367b7100.dcm (deflated 56%)
  adding: small_rsna_d